In [29]:
from causalnex.structure import StructureModel
from causalnex.network import BayesianNetwork
import pandas as pd
from causalnex.evaluation import roc_auc


In [30]:
def get_auc(sm1,df_train,df_test): 
    bn = BayesianNetwork(sm1)
    bn = bn.fit_node_states_and_cpds(df_train)


    roc, auc = roc_auc(bn, df_test, "custom_skip")
    return auc

In [21]:

df = pd.read_csv('../../data/session-df-ak.csv')
cols1 = [ 'hour_of_day', 'session_position', 'premium', 'session_length','hist_user_behavior_is_shuffle', 'custom_skip'] # , 'custom_pause_before_play'
df1 = df[cols1]
for col in cols1:
    df1[col] = df1[col].astype(int)
df1.rename(columns={'hist_user_behavior_is_shuffle':'shuffle'},inplace =True)
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df1, test_size=0.2)


c:\Users\skadi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\Users\skadi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:4298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
# GIES 
sm = StructureModel()
sm.add_edge("shuffle","session_length")
sm.add_edge("shuffle","custom_skip")
sm.add_edge("shuffle","session_position")
sm.add_edge("shuffle","premium")
sm.add_edge("session_length","session_position")
sm.add_edge("session_length","premium")
sm.add_edge("session_length","custom_skip")
sm.add_edge("premium","custom_skip")
sm.add_edge("hour_of_day","session_length")




auc_gies = get_auc(sm,df_train,df_test)
print(auc_gies)

c:\Users\skadi\AppData\Local\Programs\Python\Python37\lib\site-packages\pgmpy\models\BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  FutureWarning,


0.6707561589228699


In [35]:
sm_lingam = StructureModel()
sm_lingam.add_edge("shuffle","session_position")
sm_lingam.add_edge("shuffle","custom_skip")
sm_lingam.add_edge("session_length","premium")
sm_lingam.add_edge("session_length","custom_skip")
sm_lingam.add_edge("session_position","custom_skip")
sm_lingam.add_edge("session_position","custom_skip")
auc_lingam = get_auc(sm_lingam,df_train,df_test)
print(auc_lingam)


c:\Users\skadi\AppData\Local\Programs\Python\Python37\lib\site-packages\pgmpy\models\BayesianModel.py:10: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  FutureWarning,
c:\Users\skadi\AppData\Local\Programs\Python\Python37\lib\site-packages\causalnex\network\network.py:367: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].map(self._node_states[col])
c:\Users\skadi\AppData\Local\Programs\Python\Python37\lib\site-packages\pgmpy\factors\discrete\CPD.py:332: RuntimeWarning: invalid value encountered in true_divide
  tabular_cpd.values = (cpd / cpd.sum(axis=0)).reshape(tabular_cpd.cardinality)


0.6731249195279381


In [32]:
df_train

,hour_of_day,session_position,premium,session_length,shuffle,custom_skip
129177,1,10,0,20,1,0
41196,14,12,1,18,0,1
124439,10,14,1,15,0,1
120097,18,7,1,20,1,1
61247,3,11,0,15,0,1
...,...,...,...,...,...,...
71168,0,1,0,15,0,0
129989,12,3,1,20,0,1
100319,8,8,1,20,0,1
58142,0,5,0,15,1,1
